To resolve the comment: <br/>
Train the autoencoder + classifier for one sensor, then align the latent space of the other to the known latent space, examine the classifier result. (probably with addition/reduced texture classes) <br/>
First case: train BioTac (T4-unspv), then load the model to obtain the "known latent space"

In [1]:
import os,sys
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../new_iteration/")
import pickle
import argparse
from sklearn.metrics import confusion_matrix
from datetime import datetime

from vrae.vrae import VRAEC
from vrae.utils import *
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data as data2
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime
from tas_utils_bs import get_trainValLoader, get_testLoader

from torchsummary import summary


In [2]:
# dummy class to replace argparser
class Args:
  kfold = 0
  reduction = 1
  cuda = '0'

args=Args()

In [3]:
print("load {} kfold number, reduce data to {} folds, put to cuda:{}".format(args.kfold, args.reduction, args.cuda))

# Set hyper params
kfold_number = args.kfold
data_reduction_ratio = args.reduction
shuffle = True
num_class = 20
sequence_length_B = 400
sequence_length_I = 75
number_of_features_B = 19
number_of_features_I = 60

hidden_size = 90
hidden_layer_depth = 1
latent_length = 40
batch_size = 32
learning_rate = 0.001
n_epochs = 200


dropout_rate = 0.2
cuda = True # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
header_B = None
header_I = "CNN"

w_mse = 1 # mse between latent vectors
w_rB = 0.05 # recon for B
w_rI = 0.01 # recon for I
w_cB = 1 # classify for B
w_cI = 1 # classify for I


np.random.seed(1)
torch.manual_seed(1)

load 0 kfold number, reduce data to 1 folds, put to cuda:0


In [4]:
# Load data
data_dir = '../../new_data_folder/'

logDir = 'models_and_stats/'
# pretrained model
model_name_B = "T5-half-spv-B"
# new model 
model_name_I = "IcubCNN_ae_1_rm_0_wrI_0.01_wC_1_0"
if torch.cuda.is_available():
    device = torch.device("cuda:{}".format(args.cuda))
else:
    device = torch.device('cpu')
print("Loading data...")

# manually change the test data
kfold_number = 1
train_loader, val_loader, train_dataset, val_dataset = get_trainValLoader(data_dir, k=kfold_number, spike_ready=False, batch_size=batch_size, shuffle=shuffle)
test_loader, test_dataset = get_testLoader(data_dir, spike_ready=False, batch_size=batch_size, shuffle=shuffle)


Loading data...


In [5]:
model_B = VRAEC(num_class=num_class,
            sequence_length=sequence_length_B,
            number_of_features = number_of_features_B,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_B,
            header=header_B,
            device = device)
model_B.to(device)


model_I_pretrained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_I,
            number_of_features = number_of_features_I,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_I,
            header=header_I,
            device = device)
model_I_pretrained.to(device)

model_I_pretrained_dir = logDir+model_name_I+'.pt'

if device  == torch.device('cpu'):
    model_I_pretrained.load_state_dict(torch.load(model_I_pretrained_dir, map_location=torch.device('cpu')))
else:
    model_I_pretrained.load_state_dict(torch.load(model_I_pretrained_dir))

model_I_pretrained.eval()

print("load model I from")
print(model_name_I)

load model I from
IcubCNN_ae_1_rm_0_wrI_0.01_wC_1_0


/home/ruihan/anaconda3/envs/opencv/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



In [6]:
criterion = nn.CrossEntropyLoss()
optimB = optim.Adam(model_B.parameters(), lr=learning_rate)
cl_loss_fn = nn.NLLLoss()
recon_loss_fn = nn.MSELoss()

In [7]:
# one stage training: with recon_loss and mse_loss
training_start=datetime.now()

epoch_train_loss_B = []
epoch_train_acc_B = []
epoch_val_loss_B = []
epoch_val_acc_B = []
max_val_acc_B = 0

# epoch_train_loss_I = []
epoch_train_acc_I = []
# epoch_val_loss_I = []
epoch_val_acc_I = []
max_val_acc_I = 0

epoch_val_loss_tot = []
epoch_train_loss_tot = []

for epoch in range(n_epochs):

    # TRAIN
    model_B.train()
    model_I_pretrained.eval()
#     model_I.train()

    correct_B = 0
    train_loss_B = 0
    correct_I = 0
    train_loss_I = 0
    train_loss_tot = 0
    train_num = 0
            
    for i, (XI, XB,  y) in enumerate(train_loader):
        XI, XB, y = XI.to(device), XB.to(device), y.long().to(device)
        
        if XI.size()[0] != batch_size:
#             print("batch {} size {} < {}, skip".format(i, x.size()[0], batch_size))
            break

        train_num += XI.size(0)
        
        # train model_B
        optimB.zero_grad()  
        x_decoded_B, latent_B, output = model_B(XB)
        
        # construct loss function
        recon_loss_B = recon_loss_fn(x_decoded_B, XB)
        cl_loss_B = cl_loss_fn(output, y)
        loss_B = w_rB*recon_loss_B + w_cB*cl_loss_B
        
        # compute classification acc
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct_B += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()
        # accumulator
        train_loss_B += loss_B.item()
        
        
        # train modelI
#         optimI.zero_grad()  
        x_decoded_I, latent_I, output = model_I_pretrained(XI)
        
        # construct loss function
#         recon_loss_I = recon_loss_fn(x_decoded_I, XI)
#         cl_loss_I = cl_loss_fn(output, y)
#         loss_I = w_rI*recon_loss_I # + w_cI*cl_loss_I

        # compute classification acc
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct_I += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()
        # accumulator
#         train_loss_I += loss_I.item()
        
        loss_C = F.mse_loss(latent_B, latent_I)
        if epoch<20:
            loss = loss_B
        else:
            loss =  loss_B + w_mse*loss_C 
        
#         if epoch < 20:
# #             loss_I.backward()
#             loss_B.backward()
#         else:
#             loss.backward()
        loss.backward()
        
        train_loss_tot += loss.item()
        
        optimB.step() 
#         optimI.step() 
    
    if epoch < 20 or epoch%200 == 0:
        print("last batch training: LB: {:.2f}, LC: {:.2f}, recon_B {:.2f}, cl_B {:.2f}"\
              .format(loss_B, loss_C, recon_loss_B, cl_loss_B))
    
    
    # fill stats
    train_accuracy_B = correct_B / train_num # len(train_loader.dataset)
    train_loss_B /= train_num #len(train_loader.dataset)
    epoch_train_loss_B.append(train_loss_B)
    epoch_train_acc_B.append(train_accuracy_B) 
    
    train_accuracy_I = correct_I / train_num # len(train_loader.dataset)
#     train_loss_I /= train_num #len(train_loader.dataset)
#     epoch_train_loss_I.append(train_loss_I)
    epoch_train_acc_I.append(train_accuracy_I) 
    
    epoch_train_loss_tot.append(train_loss_tot)
    

    # VALIDATION
    model_B.eval()
    model_I_pretrained.eval()

    correct_B = 0
    val_loss_B = 0
    correct_I = 0
    val_loss_I = 0
    val_loss_tot = 0
    val_num = 0

    for i, (XI, XB,  y) in enumerate(val_loader):
        XI, XB, y = XI.to(device), XB.to(device), y.long().to(device)
        
        if XI.size()[0] != batch_size:
#             print("batch {} size {} < {}, skip".format(i, x.size()[0], batch_size))
            break

        val_num += XI.size(0)
        
        # eval model_B
        x_decoded_B, latent_B, output = model_B(XB)
        # construct loss function
        recon_loss_B = recon_loss_fn(x_decoded_B, XB)
        cl_loss_B = cl_loss_fn(output, y)
        loss_B = w_rB*recon_loss_B + w_cB*cl_loss_B
       
        # compute classification acc
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct_B += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()
        # accumulator
        val_loss_B += loss_B.item()
        
        
        # eval modelI 
        x_decoded_I, latent_I, output = model_I_pretrained(XI)
        # construct loss function
#         recon_loss_I = recon_loss_fn(x_decoded_I, XI)
#         cl_loss_I = cl_loss_fn(output, y)
#         loss_I = w_rI*recon_loss_I # + w_cI*cl_loss_I

        # compute classification acc
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct_I += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()
        # accumulator
#         val_loss_I += loss_I.item()
        
        loss_C = F.mse_loss(latent_B, latent_I)
        loss =  loss_B + w_mse*loss_C
        val_loss_tot += loss.item()

    # fill stats
    val_accuracy_B = correct_B / val_num # len(train_loader.dataset)
    val_loss_B /= val_num #len(train_loader.dataset)
    epoch_val_loss_B.append(val_loss_B)
    epoch_val_acc_B.append(val_accuracy_B) 
    
    val_accuracy_I = correct_I / val_num # len(train_loader.dataset)
#     val_loss_I /= val_num #len(train_loader.dataset)
#     epoch_val_loss_I.append(val_loss_I)
    epoch_val_acc_I.append(val_accuracy_I) 
    epoch_val_loss_tot.append(val_loss_tot)
    
    if epoch < 20 or epoch%200 == 0:
#         print("train_num {}, val_num {}".format(train_num, val_num))
        print("Epoch {}: Loss: lc {:.3f},  train_B{:.3f}, val_B {:.3f}, \n\t Acc: train_B {:.3f}, val_B {:.3f}, train_I {:.3f}, val_I {:.3f}"\
              .format(epoch, loss_C, train_loss_B, val_loss_B, train_accuracy_B, val_accuracy_B, train_accuracy_I, val_accuracy_I))
        print("-"*20)

    # choose model
    # TODO: not save at the same time, may have bad common representation
    if max_val_acc_B <= val_accuracy_B:
        model_dir = logDir + model_name_B + '.pt'
        print("Saving model at {} epoch to {}".format(epoch, model_dir))
        max_val_acc_B = val_accuracy_B
        torch.save(model_B.state_dict(), model_dir)

#     if max_val_acc_I <= val_accuracy_I:
#         model_dir = logDir + model_name_I + '.pt'
#         print("Saving model at {} epoch to {}".format(epoch, model_dir))
#         max_val_acc_I = val_accuracy_I
#         torch.save(model_I.state_dict(), model_dir)
    

training_end =  datetime.now()
training_time = training_end -training_start 
print("RAE training takes time {}".format(training_time)) 

last batch training: LB: 19.47, LC: 7.44, recon_B 335.80, cl_B 2.68
Epoch 0: Loss: lc 7.550,  train_B0.687, val_B 0.665, 
	 Acc: train_B 0.155, val_B 0.443, train_I 0.977, val_I 1.000
--------------------
Saving model at 0 epoch to models_and_stats/T5-half-spv-B.pt
last batch training: LB: 16.59, LC: 7.76, recon_B 288.65, cl_B 2.16
Epoch 1: Loss: lc 7.541,  train_B0.588, val_B 0.540, 
	 Acc: train_B 0.472, val_B 0.490, train_I 0.977, val_I 1.000
--------------------
Saving model at 1 epoch to models_and_stats/T5-half-spv-B.pt
last batch training: LB: 15.79, LC: 8.02, recon_B 283.30, cl_B 1.63
Epoch 2: Loss: lc 8.431,  train_B0.499, val_B 0.462, 
	 Acc: train_B 0.535, val_B 0.641, train_I 0.979, val_I 1.000
--------------------
Saving model at 2 epoch to models_and_stats/T5-half-spv-B.pt
last batch training: LB: 12.90, LC: 8.96, recon_B 232.68, cl_B 1.26
Epoch 3: Loss: lc 9.178,  train_B0.433, val_B 0.415, 
	 Acc: train_B 0.604, val_B 0.688, train_I 0.977, val_I 1.000
------------------

In [9]:
model_B.is_fitted = True
model_I_pretrained.is_fitted = True

model_B.eval()
model_I_pretrained.eval()

VRAE(n_epochs=200,batch_size=32,cuda=False)

In [11]:
# copy the classifier from B to I to examine the classification result
classifier_keys = ['classifier.0.weight', 'classifier.0.bias']
classifier_dict_B = {k: v for k, v in model_B.state_dict().items() if k in classifier_keys}
classifier_dict_I = {k: v for k, v in model_I_pretrained.state_dict().items() if k in classifier_keys}
ae_dict_B = {k: v for k, v in model_B.state_dict().items() if k not in classifier_keys}
ae_dict_I = {k: v for k, v in model_I_pretrained.state_dict().items() if k not in classifier_keys}

newB_dict = model_B.state_dict()
newB_dict.update(ae_dict_B)
# overwrite classifer for new models
newB_dict.update(classifier_dict_I)
# load the new state_dict
model_B.load_state_dict(newB_dict)
model_B.to(device)
model_B.eval()
print(model_B)

VRAE(n_epochs=200,batch_size=32,cuda=False)


In [13]:
# TEST
correct_B = 0
correct_I = 0
test_num = 0

for i, (XI, XB,  y) in enumerate(test_loader):
    XI, XB, y = XI.to(device), XB.to(device), y.long().to(device)

    if XI.size()[0] != batch_size:
#             print("batch {} size {} < {}, skip".format(i, x.size()[0], batch_size))
        break

    test_num += XI.size(0)

    # test model_B
    x_decoded_B, latent_B, output = model_B(XB)

    # compute classification acc
    pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
    correct_B += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()

    
    # test modelI 
    x_decoded_I, latent_I, output = model_I_pretrained(XI)

    # compute classification acc
    pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
    correct_I += pred.eq(y.data.view_as(pred)).long().cpu().sum().item()

test_acc_B = correct_B/test_num
test_acc_I = correct_I/test_num
print('Test accuracy for {} fold {} samples: B {}, I {}'.format(str(kfold_number),test_num, test_acc_B, test_acc_I))

Test accuracy for 1 fold 192 samples: B 0.9479166666666666, I 0.9270833333333334


In [ ]:
# save stats
# results_dict = {"epoch_train_loss_B": epoch_train_loss_B,
#                 "epoch_train_loss_I": epoch_train_loss_I,
#                 "epoch_val_loss_B": epoch_val_loss_B,
#                 "epoch_val_loss_I": epoch_val_loss_I,
#                 "epoch_train_acc_B": epoch_train_acc_B,
#                 "epoch_train_acc_I": epoch_train_acc_I,
#                 "epoch_val_acc_B": epoch_val_acc_B,
#                 "epoch_val_acc_I": epoch_val_acc_I,
#                 "test_acc": [test_acc_B, test_acc_I]}

# since we examine the classifier at the last step, the train acc and val acc do not matter
results_dict = {"epoch_train_loss_B": epoch_train_loss_B,
                "epoch_val_loss_B": epoch_val_loss_B,
                "epoch_train_loss_tot": epoch_train_loss_tot,
                "epoch_val_loss_tot": epoch_val_loss_tot,
#                 "epoch_train_acc_B": epoch_train_acc_B,
#                 "epoch_train_acc_I": epoch_train_acc_I,
#                 "epoch_val_acc_B": epoch_val_acc_B,
#                 "epoch_val_acc_I": epoch_val_acc_I,
                "test_acc": [test_acc_B, test_acc_I]}
dict_name = "T5_BT19Icub_joint_ae_wrB_{}_wcB_{}_wrI_{}_wcI_{}_wC_{}_{}_trainB.pkl".format(w_rB, w_cB, w_rI, w_cI, w_mse, str(kfold_number))
pickle.dump(results_dict, open(logDir + dict_name, 'wb'))
print("dump results dict to {}".format(dict_name))


In [ ]:
# plot loss
assert n_epochs == len(epoch_train_loss_tot), "different epoch length {} {}".format(n_epochs, len(epoch_train_loss_tot))
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(np.arange(n_epochs), epoch_train_loss_tot, label="epoch_train_loss_tot")
ax.set_xlabel('epoch')
ax.set_ylabel('acc')
ax.grid(True)
plt.legend(loc='upper right')
figname = logDir + model_name_B + "_train_loss_tot.png"
plt.savefig(figname)
plt.show()
